In [39]:
import argparse
import time
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from scipy import sparse
from dataset import *
from dataloader import *
from loss import *
from model import *
from torch.nn import Softmax
import os
import bottleneck as bn
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [22]:
if torch.cuda.is_available():
        cuda_aval = True

device = torch.device("cuda" if cuda_aval else "cpu")
print("Now using device : ", device)
###################
print("Load and Preprocess Movielens dataset")
# Load Data
uim = pd.read_csv("/opt/ml/input/workspace/CF/Non-DL/FISM/user_movie_interaction.csv")
item_id = np.array(list(uim)[1:])
user_id = uim["user"].to_numpy()
uim_np = uim.to_numpy()[:, 1:]
print("Done!")

Now using device :  cuda
Load and Preprocess Movielens dataset
Done!


In [23]:
#load model
f = open("model.pt", 'rb')
model  = torch.load(f).to(device)

In [52]:
model.eval()
result = list()
for i in range(31360) : 
    user = user_id[i]
    data_np = uim_np[i]
    data = torch.FloatTensor(data_np)
    data = data.reshape(1,-1).to(device)
    
    probability, _, _ = model(data)
    probability_np = np.array(probability.cpu().detach())[0]
    
    probability_np[data_np.nonzero()] = -np.inf

    idx = np.argpartition(probability_np, -10)[-10:]
    items = item_id[idx]
    for item in items :
        result.append((user, item))
    if i%5000 == 0 :
        print(f"{i}/31360 complete")

[1.6077598 1.5373878 2.0355206 1.6266432 1.5519786 1.6021719 1.5140964
 1.5378928 1.4829865 1.4856646]
['915' '520' '83798' '216' '2696' '1078' '1617' '1884' '7178' '1232']


/opt/conda/lib/python3.8/site-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


In [44]:
info = pd.DataFrame(result, columns=['user','item'])
info.to_csv("submission.csv",index=False)
print("Inference Done!")

KeyboardInterrupt: 